# Hybrid Images

## getting fourier transform

In [23]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

#reading images and get fourier transform and shift them
near_image = cv2.imread('near03.jpg')
near_image_gray = cv2.cvtColor(near_image, cv2.COLOR_BGR2GRAY)
f_near_image = np.fft.fft2(near_image_gray)
f_near_shift = np.fft.fftshift(f_near_image)
far_image = cv2.imread('far04.jpg')
far_image_gray = cv2.cvtColor(far_image, cv2.COLOR_BGR2GRAY)
f_far_image = np.fft.fft2(far_image_gray)
f_far_shift = np.fft.fftshift(f_far_image)

#save log of images
cv2.imwrite('near_dft_05.jpg', 20 * np.log(np.abs(f_near_shift)))
cv2.imwrite('far_dft_06.jpg', 20 * np.log(np.abs(f_far_shift)))

True

## gassian filtering

In [ ]:
#function for gaussian filter
def gaussian_filter(image, sigma, highpass):
    y, x = image.shape
    gauss_filter = np.zeros((y, x), dtype=complex)
    for j in range(y):
        for i in range(x):
            if highpass:
                gauss_filter[j, i] = image[j, i] * (1 - math.exp(-((i - x//2)**2+(j - y//2)**2) / (2*sigma**2)))
            else:
                gauss_filter[j, i] = image[j, i] * math.exp(-((i - x//2)**2+(j - y//2)**2) / (2*sigma**2))
                
    return gauss_filter

#filtering images
s = 200
r = 100
f_near_filter = gaussian_filter(f_near_shift, r, True)
f_far_filter = gaussian_filter(f_far_shift, s, False)

cv2.imwrite('highpass07.jpg', np.abs(f_near_filter))
cv2.imwrite('lowpass08.jpg', np.abs(f_far_filter))

## cutting off

In [6]:
def cutoff(image, value, highpass):
    new_image = image
    y, x = image.shape
    for i in range(x):
        for j in range(y):
            if highpass:
                if (i - x//2)**2 + (j - y//2)**2 <= value**2:
                    new_image[j, i] = 0
            else:
                if (i - x//2)**2 + (j - y//2)**2 >= value**2:
                    new_image[j, i] = 0
    return new_image
near_cutoff = 5
far_cutoff = 10
cutoff_near = cutoff(f_near_filter, near_cutoff, True)
cutoff_far = cutoff(f_far_filter, far_cutoff, False)

cv2.imwrite('highpassed09.jpg', np.abs(cutoff_near))
cv2.imwrite('lowpassed10.jpg', np.abs(cutoff_far))

True

## Combination of two images

In [8]:
y, x = cutoff_near.shape
final_image = np.zeros(cutoff_near.shape, dtype = complex)
a, b = 1, 1
for j in range(y):
    for i in range(x):
        if (j - y//2)**2 + (i - x//2)**2 <= near_cutoff**2:
            final_image[j, i] = cutoff_far[j, i]
        elif (j - y//2)**2 + (i - x//2)**2 >= far_cutoff**2:
            final_image[j, i] = cutoff_near[j, i]
        else:
            final_image[j, i] = (a * cutoff_near[j, i] + b * cutoff_far[j, i]) / (a + b)
cv2.imwrite('freauency_hybrid11.jpg', np.abs(final_image))
final_image = np.fft.ifftshift(final_image)
final_image = np.fft.ifft2(final_image)
final_image = np.abs(final_image)
cv2.imwrite('final_near.jpg', final_image)

True